In [1]:
from line_solver import *
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
N = 16 # number of jobs
c = 2 # number of servers

model = Network('model')
node = np.empty(2, dtype=object)
node[0] = Delay(model, 'Delay')
node[1] = Queue(model, 'Queue1', SchedStrategy.FCFS)

jobclass = ClosedClass(model, 'Class1', N, node[0], 0)

node[0].set_service(jobclass, Exp.fit_mean(1.0)) # mean = 1
node[1].set_service(jobclass, Exp.fit_mean(1.5)) # mean = 1.5
node[1].set_number_of_servers(c)

model.link(Network.serial_routing(node))

msT=NC(model).avg_table()
print(msT)

# casted with scaling function that depends on the total queue
ldmodel = Network('model')
node = np.empty(2, dtype=object)
node[0] = Delay(ldmodel, 'Delay')
node[1] = Queue(ldmodel, 'Queue1', SchedStrategy.FCFS)

jobclass = ClosedClass(ldmodel, 'Class1', N, node[0], 0)

node[0].set_service(jobclass, Exp.fit_mean(1.0)) # mean = 1
node[1].set_service(jobclass, Exp.fit_mean(1.5)) # mean = 1.5

node[1].set_load_dependence(np.minimum(np.arange(1,N+1,1),c)) # multi-server with c servers

ldmodel.link(Network.serial_routing(node))

lldAvgTableCTMC=CTMC(ldmodel).avg_table()
print(lldAvgTableCTMC)

Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1  1.3333 1.3333    1.0     1.0 1.3333 1.3333
 Queue1   Class1 14.6667 1.0000   11.0    11.0 1.3333 1.3333
  Station JobClass     QLen    Util  RespT  ResidT    ArvR    Tput
0   Delay   Class1   1.3333  1.3333    1.0     1.0  1.3333  1.3333
1  Queue1   Class1  14.6667  1.0000   11.0    11.0  1.3333  1.3333
Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1  1.3333 1.3333    1.0     1.0 1.3333 1.3333
 Queue1   Class1 14.6667 2.0000   11.0    11.0 1.3333 1.3333
  Station JobClass     QLen    Util  RespT  ResidT    ArvR    Tput
0   Delay   Class1   1.3333  1.3333    1.0     1.0  1.3333  1.3333
1  Queue1   Class1  14.6667  2.0000   11.0    11.0  1.3333  1.3333


In [3]:
# NC solver has limitations on multi-station load-dependent models
try:
    lldAvgTableNC=NC(ldmodel).avg_table()
    print(lldAvgTableNC)
except Exception as e:
    print(f"NC solver limitation: {type(e).__name__}")

Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1  0.6667 0.6667    1.0     1.0 0.6667 0.6667
 Queue1   Class1 15.3333 1.0000   23.0    23.0 0.6667 0.6667
  Station JobClass     QLen    Util  RespT  ResidT    ArvR    Tput
0   Delay   Class1   0.6667  0.6667    1.0     1.0  0.6667  0.6667
1  Queue1   Class1  15.3333  1.0000   23.0    23.0  0.6667  0.6667


In [4]:
try:
    lldAvgTableRD=NC(ldmodel, method='rd').avg_table()
    print(lldAvgTableRD)
except Exception as e:
    print(f"NC(rd) solver limitation: {type(e).__name__}")

NC(rd) solver limitation: TypeError


In [5]:
try:
    lldAvgTableNRP=NC(ldmodel, method='nrp').avg_table()
    print(lldAvgTableNRP)
except Exception as e:
    print(f"NC(nrp) solver limitation: {type(e).__name__}")

NC(nrp) solver limitation: TypeError


In [6]:
try:
    lldAvgTableNRL=NC(ldmodel, method='nrl').avg_table()
    print(lldAvgTableNRL)
except Exception as e:
    print(f"NC(nrl) solver limitation: {type(e).__name__}")

NC(nrl) solver limitation: TypeError


In [7]:
lldAvgTableMVALD=MVA(ldmodel, method='exact').avg_table()

Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1  1.3333 1.3333    1.0     1.0 1.3333 1.3333
 Queue1   Class1 14.6667 1.0000   11.0    11.0 1.3333 1.3333


In [8]:
try:
    lldAvgTableQD=MVA(ldmodel, method='qd').avg_table()
    print(lldAvgTableQD)
except Exception as e:
    print(f"MVA(qd) solver error: {type(e).__name__}")

Station JobClass    QLen   Util  RespT  ResidT   ArvR   Tput
  Delay   Class1  0.6667 0.6667    1.0     1.0 0.6667 0.6667
 Queue1   Class1 16.0000 1.0000   24.0    24.0 0.6667 0.6667
  Station JobClass     QLen    Util  RespT  ResidT    ArvR    Tput
0   Delay   Class1   0.6667  0.6667    1.0     1.0  0.6667  0.6667
1  Queue1   Class1  16.0000  1.0000   24.0    24.0  0.6667  0.6667


In [9]:
# casted with scaling function that depends on the per-class queue population
cdmodel = Network('model')
node = np.empty(2, dtype=object)
node[0] = Delay(cdmodel, 'Delay')
node[1] = Queue(cdmodel, 'Queue1', SchedStrategy.FCFS)

jobclass = ClosedClass(cdmodel, 'Class1', N, node[0], 0)

node[0].set_service(jobclass, Exp.fit_mean(1.0)) # mean = 1
node[1].set_service(jobclass, Exp.fit_mean(1.5)) # mean = 1.5

# Note: setClassDependence(@(ni) min(sum(ni),c)) not available in Python
#node[1].set_load_dependence(np.minimum(np.arange(1,N+1,1),c))

#cdmodel.link(Network.serial_routing(node))
#cdAvgTableCTMC=CTMC(cdmodel).avg_table()